In [1]:
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
#Vertical resolution used : 
0.02
0.1
0.5
1.25010428797
8.33294983235
32.8082236775

32.8082236775

In [3]:
import sys, os

sys.path.append(os.path.join(os.environ['ATS_SRC_DIR'],'tools','meshing_ats'))
import meshing_ats
reload(meshing_ats)
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import scipy.signal

In [4]:
xl =pd.ExcelFile('Topo_Profiles_Barrow.xlsx')

In [5]:
xl.sheet_names
print (xl.sheet_names)
#data = xl.parse(sheet_name='Transect B, Polygon near ERT')
#data = xl.parse(sheet_name='Transect Area B')
data = xl.parse(sheet_name='Transect B, Area C')
keys = data.keys()
slice_at_right = False

[u'Transect A, Polygon near ERT', u'Transect B, Polygon near ERT', u'Transect Area B', u'Transect A, Area C', u'Transect B, Area C', u'Repeat ERT Transect']


In [6]:
plt.plot(data['Distance_m'], data['Z'])
plt.ylabel('Elevation')
plt.title('Topography, Polygon Area C')
plt.show()

In [7]:
# smooth the topographic data : September 24
length = len(data['Distance_m'])
x = []
z = []

if (slice_at_right):
    x = np.array(data['Distance_m'][length/2:])
    z = np.array(data['Z'][length/2:])
else:
    x = np.array(data['Distance_m'][:46])
    z = np.array(data['Z'][:46])

x1 = [x[0], x[10]]
z1 = [z[0], z[10]]
x2 = [x[10], x[22], x[-1]]
z2 = [z[10], z[22], z[-1]]
cs1 = scipy.interpolate.CubicSpline(x1,z1, bc_type='clamped')
cs2 = scipy.interpolate.CubicSpline(x2,z2, bc_type='clamped')
z_left = np.concatenate((cs1(x[:11]), cs2(x[11:])))
#z_smooth = cs1(x)
if (slice_at_right):
    z = z[::-1]
    z_left = z_left[::-1]
    x = x[::-1]

#print (len(z_smooth), z_smooth[len(z_smooth)-1], min(z_smooth), max(z_smooth))

xx = x1 + x2
zz = z1 + z2

plt.plot(x, z, '.')
plt.plot(xx,zz , '*')
plt.plot(x, z_left, '-', color = 'g')
plt.ylabel('Elevation')
plt.title('Mesh, Polygon Area C')
plt.show()


5.01
[4.81504 4.78504 4.73504 4.68504 4.63504 4.58504 4.53504 4.48504 4.43504
 4.33504 4.23504 4.13504 4.03504 3.83504 3.58504 3.33504]
4.8772 [0.] 4.83503993225476


In [8]:
import math
def Distance(x,y):
    d = np.zeros(len(x))
    for i in range(len(x)):
        d[i] = math.hypot(x[-1] - x[i], y[-1] - y[i])
    return d

In [9]:
#Vladimir data
site_area = 'C'
if site_area == 'C': 
    print site_area
    zv = np.array([5.099, 5.169, 5.321, 4.968])
    zv =zv[::-1]
    northing = [7910344.103,7910341.163,7910337.608, 7910335.845] #original
    easting = [585907.153,585907.605,585908.893,585909.606]
    distC = Distance(northing,easting)
    distC = distC[::-1]
    
    #print distC, zv
    numx1 = int((distC[1] - distC[0]) / 0.22)
    numx2 = int((distC[-1] - distC[1]) / 0.24)
    print numx1, numx2
    xnew1 = np.linspace(distC[0],distC[1],numx1)
    xnew2 = np.linspace(distC[1],distC[-1],numx2)
    cs1 = scipy.interpolate.CubicSpline(distC[:2],zv[:2], bc_type='clamped')
    cs2 = scipy.interpolate.CubicSpline(distC[1:],zv[1:], bc_type='clamped')

plt.plot(xnew1, cs1(xnew1), '.-', color= 'r', label = 'fitted curve')
plt.plot(xnew2, cs2(xnew2), '.-', color= 'r')
plt.plot(distC, zv, '*', color= 'k', label='measured data')
#plt.plot(x, z_left, 'k-')

plt.show()


xnew = np.concatenate((xnew1,xnew2[1:]))
zv = np.concatenate((cs1(xnew1), cs2(xnew2[1:])))
z_right = np.array(zv)
print ('---------------------------------------------')

assert len(zv) == len(xnew)

#print xnew1
#print xnew2
#print len(xnew)
#
plt.xlabel('Distance [m]')
plt.ylabel('Elevation [m]')
plt.title('Polygon Area B')
plt.legend()


C
8 27
---------------------------------------------


In [28]:
print z_left[-1]
print z_right[0],z_right[-1]
d = z_right[-1] - z_left[-1]
#z_left = z_left + d
#print x
#print xnew
X = []
for i in xnew[1:]:
        #print i
        X.append(x[-1] + i)

#print X
X = np.array(X)
X = np.concatenate((x,X))
Z = np.concatenate((z_left[:-1] ,z_right[::-1]))
Z1 = np.concatenate((z_left[:-1] +d,z_right[::-1]))
print Z1, len(z_left), len(z_right), len(Z1)
assert len(Z) == len(X)
plt.plot(X,Z, '.')
plt.plot(X,Z1, '*-',color='r')
plt.plot(X[int(79/2)], Z1[int(79/2)],'^', color='g')
assert len(Z)%2 == 1
radius_pos = int(79/2)
#radius_pos = int(len(Z)/2)
print X[int(79/2)], X[-1]
print radius_pos, len(Z), d

4.920000076290001
4.968 5.099
[5.00899985 5.01403986 5.02771988 5.04787991 5.07235995 5.099
 5.12564005 5.15012009 5.17028012 5.18396014 5.18900015 5.18839842
 5.18668576 5.184001   5.18048298 5.1762705  5.17150241 5.16631752
 5.16085465 5.15525264 5.1496503  5.14418647 5.13899996 5.13420359
 5.12980609 5.12579019 5.12213861 5.11883407 5.1158593  5.11319701
 5.11082993 5.10874078 5.10691229 5.10532717 5.10396815 5.10281795
 5.10185929 5.1010749  5.10044749 5.0999598  5.09959454 5.09933443
 5.0991622  5.09906057 5.09901226 5.099      5.09956802 5.10126227
 5.10406799 5.10797044 5.1129549  5.11900661 5.12611083 5.13425283
 5.14341786 5.15359118 5.16475806 5.17689361 5.18984373 5.20336517
 5.21721292 5.23114196 5.24490728 5.25826386 5.27096668 5.28277073
 5.293431   5.30270246 5.31034011 5.31609893 5.3197339  5.321
 5.30144606 5.25101749 5.18206414 5.10693586 5.03798251 4.98755394
 4.968     ] 46 34 79
10.7335106265 20.9994454827741
39 79 0.1789999237099993


In [27]:
#trough
observe = 'center'
xloc = 0
zv = Z1
z_smooth = Z1
mid_pt = (zv[0] + zv[1])/2.

depths_t = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
depths_r = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 0.35, 0.4, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
depths_c = [0.02, 0.05, 0.1, 0.15, 0.2, 0.25, 0.30, 00.35, 0.4, 0.5, 0.6,0.7,0.8, 1.0, 1.25, 1.50]
if observe == 'trough':
    z_location = [zv[-1] - v for v in depths_t] #taking z at zv[1] to make sure the observed point lie within above cell
    xloc = np.where(zv == zv[0])
    xloc = x[xloc]
elif observe == 'rim':
    z_location = [max(zv) - v for v in depths_r]
    xloc = np.where(zv == max(zv))
elif observe == 'center':
    z_location = [zv[radius_pos] - v for v in depths_c]
print round(z_smooth[10],5)
print np.round(z_location,5)
print  xloc, len(zv)


5.189
[5.07996 5.04996 4.99996 4.94996 4.89996 4.84996 4.79996 4.74996 4.69996
 4.59996 4.49996 4.39996 4.29996 4.09996 3.84996 3.59996]
0 79


In [15]:
reload(meshing_ats)

#m2 = meshing_ats.Mesh2D.from_Transect_Make3DPie1(X,Z1, radius_pos, curved=False)
m2 = meshing_ats.Mesh2D.from_Transect_Make3DUp(X,Z1)
m2.plot()


-0.4658308147248193 -0.4870477038868227
[ 0.          0.27521822  0.55043644  0.82565466  1.10087288  1.37609111
  1.65130933  1.92652755  2.20174577  2.47696399  2.75218221  3.02740043
  3.30261865  3.57783688  3.8530551   4.12827332  4.40349154  4.67870976
  4.95392798  5.2291462   5.50436442  5.77958265  6.05480087  6.33001909
  6.60523731  6.88045553  7.15567375  7.43089197  7.70611019  7.98132841
  8.25654664  8.53176486  8.80698308  9.0822013   9.35741952  9.63263774
  9.90785596 10.18307418 10.45829241 10.73351063] [ 0.          0.27521822  0.55043644  0.82565466  1.10087288  1.37609111
  1.65130933  1.92652755  2.20174577  2.47696399  2.75218221  3.02740043
  3.30261865  3.57783688  3.8530551   4.12827332  4.40349154  4.67870976
  4.95392798  5.2291462   5.50436442  5.77958265  6.05480087  6.33001909
  6.60523731  6.88045553  7.15567375  7.43089197  7.70611019  7.98132841
  8.25654664  8.53176486  8.80698308  9.0822013   9.35741952  9.63263774
  9.90785596 10.18307418 10.458292

In [16]:
outfile = "/Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/"
layer_types = []
layer_data = []
layer_ncells = []
layer_mat_ids = []

z=0
Z = []

for i in range(1):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1001)
    z = z + 0.02
    Z.append(z)
print ('Moss ', z)

for i in range(4):
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1002)
    z = z + 0.02
    Z.append(z)
print ('Peat ', z)

for i in range(15): 
    layer_types.append('constant')
    layer_data.append(0.02)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + 0.02
    Z.append(z)
print ('Upper mineral ', z)

dz = .02
for i in range(35):
    dz *= 1.075
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print ('Ice rich', z)

for i in range(25):
    dz *= 1.12
    layer_types.append("constant")
    layer_data.append(dz)
    layer_ncells.append(1)
    layer_mat_ids.append(1003)
    z = z + dz
    Z.append(z)
print (z)


layer_types.append('snapped')
layer_data.append(-45.0) # bottom location
layer_ncells.append(1)
layer_mat_ids.append(1003)

m3 = meshing_ats.Mesh3D.extruded_Mesh2D(m2, layer_types, 
                                        layer_data, 
                                        layer_ncells, 
                                        layer_mat_ids)


#m3.write_exodus(outfile + "areaC-updist-vladimir-om10cm.exo")
m3.write_exodus(outfile + "areaC-3layers-3D-om10cm.exo")

('Moss ', 0.02)
('Peat ', 0.1)
('Upper mineral ', 0.4000000000000001)
('Ice rich', 3.716409519999923)
41.2555869471

You are using exodus.py v 1.05 (beta-cmake), a python wrapper of some of the exodus II library.

Copyright (c) 2013, 2014, 2015, 2016, 2017 National Technology &
Engineering Solutions of Sandia, LLC (NTESS).  Under the terms of
Contract DE-NA0003525 with NTESS, the U.S. Government retains certain
rights in this software.  

Opening exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-3D-om10cm.exo
Closing exodus file: /Users/ajc/Desktop/ATS/meshes/barrow-pie-wedges/areaC-3layers-3D-om10cm.exo


In [ ]:
0.02
0.1
0.5
1.25010428797
8.33294983235
32.8082236775